## ================================================================
# Purpose:      Python script for Association Rules
# Author:       BC2407-S02 Team 8
# Updated:      03/04/2021
# Topics:       Slight Data Cleaning, Association Rules
# Data Source:  Kaggle E-commerce Data
# Packages:     pandas, numpy, mlxtend
## ================================================================


In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
df = pd.read_csv("data_final.csv", encoding = "ISO-8859-1" , index_col = 0)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


# Dataset Cleanup

Firstly, we notice some of the descriptions in the dataset have spaces to be removed.

Next, we will drop the rows with NA values under the ['InvoiceNo'] column, as well as remove credit transactions, which are InvoiceNo containing 'C' at the front.

In [3]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


# Dataset Preparation - Slicing dataset based on Country (France)

Before we perform the apriori algorithm onto this dataset, there are further steps to be taken.

Firstly, we have to group the dataset by the InvoiceNo, and add to that row, the columns of items that were purchased in that invoice.

Next, we will fill the columns that are NA with 0, indicating that these particular items are not purchased in the corresponding Invoice.

For the sake of keeping the data set small, we are only looking at sales for France. We chose to look at France because it has one of the largest sales number, only second to United Kingdom

In [4]:
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


From the 'basket' dataframe above, we can see that we have a total of 392 unique transactions, as well as 1563 unique items sold, throughout all the transactions made from France.

We can see that there are alot of 0's in the dataframe, however we have to make sure there is only 1's and 0's in the entire dataframe, which will be difficult to determine by just observing. Hence, we will complete this step by performing hot encoding of the data.

Furthermore, we will also remove the POSTAGE column as this is an irrelevant column in our exploration (this charge is regarding the fees for postage)

In [5]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

Now that we have finished structuring our data, it is time to perform the apriori algorithm to obtain frequent item sets.
In this example, we will observe item sets with a minimum support of 7%.

In [6]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

frequent_itemsets.head()

,support,itemsets
0,0.071979,(4 TRADITIONAL SPINNING TOPS)
1,0.097686,(ALARM CLOCK BAKELIKE GREEN)
2,0.102828,(ALARM CLOCK BAKELIKE PINK)
3,0.095116,(ALARM CLOCK BAKELIKE RED)
4,0.077121,(BAKING SET 9 PIECE RETROSPOT)


After obtaining the frequent item sets, we will use 'association_rules' to obtain the antecedents and consequents of the itemset, as well as their corresponding lift-support-confidence values.

In [7]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.097686,0.102828,0.074550,0.763158,7.421711,0.064505,3.788061
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102828,0.097686,0.074550,0.725000,7.421711,0.064505,3.281140
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.097686,0.095116,0.079692,0.815789,8.576814,0.070400,4.912229
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.095116,0.097686,0.079692,0.837838,8.576814,0.070400,5.564267
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.095116,0.102828,0.074550,0.783784,7.622297,0.064770,4.149422


We will now filter this dataframe. In particular, we will only look for itemsets rules that have high lift (6) and high confidence (8).

We can filter the dataframe using standard pandas code. In this case, look for a large lift (6) and high confidence (.8):

Lift helps us to measure how useful the particular rule is, in the context of the existing situation. Even though a rule may have high confidence and high support, it is still important to look at the Lift value as it indicates to us the impact of the antecedent on the consequent. In this case, a Lift value of greater than 1, will tell us that the antecedent can help to boost the occurence of the consequent.

In [8]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ].sort_values(by = 'confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
21,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612
22,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301
19,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563
18,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.095116,0.097686,0.079692,0.837838,8.576814,0.070400,5.564267
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.097686,0.095116,0.079692,0.815789,8.576814,0.070400,4.912229
20,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.123393,0.133676,0.100257,0.812500,6.078125,0.083762,4.620394


# Observations drawn from the Association Rules generated

From the dataframe above that has been sorted by descending confidence values, the e-commerce site can easily observe the clear relationship between the items sold in their shop.

Let us take the first row for example, regarding the sales of 'Red Retrospot Paper Napkins (X)','Red Spotty Paper Cups (Y)' and 'Red Spotty Paper Plates (Z)'

In particular, the rule states {X,Y} -> Z, has the highest confidence value of 0.975 and an extremely high lift value of 7.644.

This tells us that given that a customer purchases X and Y, there is a 97.5% chance that they will also purchase Z. The high lift value tells us that the sale of X and Y does indeed boost the sale of Z. 

Let us look at the total sales for X, Y and Z.

In [9]:
print("Red Retrospot Paper Napkins (X) : " ,basket['SET/20 RED RETROSPOT PAPER NAPKINS'].sum())

print("Red Spotty Paper Cups (Y) : " ,basket['SET/6 RED SPOTTY PAPER CUPS'].sum())

print("Red Spotty Paper Plates (Z) : " ,basket['SET/6 RED SPOTTY PAPER PLATES'].sum())

Red Retrospot Paper Napkins (X) :  960.0
Red Spotty Paper Cups (Y) :  1272.0
Red Spotty Paper Plates (Z) :  1116.0


# Conclusions to be drawn from the observations

In looking at the total sales of the items in the rules, we can see that there is a slight deviation in their numbers. 

This association rule thus indicates to the e-commerce shop owner that there could be something done to promote the sales for Z. In fact, if you look at the top 4 rows of the dataframe, you can see that they are all rules regarding the sale of Red Paper Napkins/Cups/Plates, and they all have high confidence and lift values. This shows to us that many customers are looking to purchase these items together, perhaps it is for party planning purposes, or for corporate events.

Extrapolating from this information, a very useful tactic that the e-commerce shop can employ is to promote a sale, for these items to be bought together. For example, they can be sold in a package, at a slightly cheaper price. In this case, customers are likely to purchase the entire package, instead of the items individually. Hence, this will boost the overall sales for all the items.

Thus, we can learn that Association Rule can play a very important part in helping e-commerce shop owners analyze what they may be missing in the purchasing patterns of their customers, and eventually help to boost their sales.

# -----------Extra exploration of Association Rules-----------

## Exploring Country == 'Germany'

Now, we will focus our association rules to transactions made for Germany.

In this example, we made use of a minimum 5% support and 50% confidence.

In [10]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)

frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)


rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)].sort_values(by = 'confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746
0,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
7,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670


## Exploring specific customers' purchasement patterns

Now, we will explore a specific Customer to find out patterns in their purchase. In particular, we have chosen to use CustomerID '14646' as this customer has the largest spending in the store, as we have identified in the Data Explorations notebook.

In [11]:
basket3 = (df[df['CustomerID'] == 14646.0] 
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets3 = basket3.applymap(encode_units)
basket_sets3.drop('POSTAGE', inplace=True, axis=1)

frequent_itemsets3 = apriori(basket_sets3, min_support=0.1, use_colnames=True)
rules3 = association_rules(frequent_itemsets3, metric="lift", min_threshold=1)

rules3[ (rules3['lift'] >= 6) &
        (rules3['confidence'] >= 0.8)].sort_values(by = 'confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
302,"(LUNCH BAG RED RETROSPOT, SPACEBOY BIRTHDAY CARD)",(PLASTERS IN TIN SPACEBOY),0.108108,0.162162,0.108108,1.000000,6.166667,0.090577,inf
539,"(LUNCH BAG RED RETROSPOT, SPACEBOY BIRTHDAY CA...",(PLASTERS IN TIN SPACEBOY),0.108108,0.162162,0.108108,1.000000,6.166667,0.090577,inf
542,"(LUNCH BAG RED RETROSPOT, SPACEBOY BIRTHDAY CARD)","(PLASTERS IN TIN SPACEBOY, ROUND SNACK BOXES S...",0.108108,0.162162,0.108108,1.000000,6.166667,0.090577,inf
177,"(LUNCH BAG RED RETROSPOT, ROUND SNACK BOXES SE...",(COWBOYS AND INDIANS BIRTHDAY CARD),0.121622,0.135135,0.108108,0.888889,6.577778,0.091673,7.783784
178,"(COWBOYS AND INDIANS BIRTHDAY CARD, ROUND SNAC...",(LUNCH BAG RED RETROSPOT),0.121622,0.135135,0.108108,0.888889,6.577778,0.091673,7.783784
304,"(SPACEBOY BIRTHDAY CARD, PLASTERS IN TIN SPACE...",(LUNCH BAG RED RETROSPOT),0.121622,0.135135,0.108108,0.888889,6.577778,0.091673,7.783784
541,"(SPACEBOY BIRTHDAY CARD, PLASTERS IN TIN SPACE...",(LUNCH BAG RED RETROSPOT),0.121622,0.135135,0.108108,0.888889,6.577778,0.091673,7.783784
544,"(LUNCH BAG RED RETROSPOT, ROUND SNACK BOXES SE...","(SPACEBOY BIRTHDAY CARD, PLASTERS IN TIN SPACE...",0.121622,0.121622,0.108108,0.888889,7.308642,0.093316,7.905405
545,"(SPACEBOY BIRTHDAY CARD, PLASTERS IN TIN SPACE...","(LUNCH BAG RED RETROSPOT, ROUND SNACK BOXES SE...",0.121622,0.121622,0.108108,0.888889,7.308642,0.093316,7.905405
179,(LUNCH BAG RED RETROSPOT),"(COWBOYS AND INDIANS BIRTHDAY CARD, ROUND SNAC...",0.135135,0.121622,0.108108,0.800000,6.577778,0.091673,4.391892


# Observations drawn from the Association Rules generated

For this particular customer, the top 3 rows can be seen to have a confidence value of 100%. This indicates to us that, for all the purchases made, as long as the antecedents occur, the consequents will always occur as well. In other words, they will always purchase the items in the antecedents and conseqeunts together.

For this example, there might be slightly different conclusions to be drawn. 

For instance, it may not necessarily make sense for the shopowner to introduce packages for the items to be sold together at a lower price in order to boost sales, because in this instance, the customer ALWAYS purchases the item regardless. Hence, promoting a package for items in the first 3 rules may decrease the profits earned instead. 

However, this package promotion could still be applied to rules that have confidence < 1.

However, there are still useful conclusions that can be drawn from these Association Rules. For instance, it is fairly common for e-commerce shops to have recommended products to the user. In this case, the e-commerce website/app can display these commonly bought items on the recommended page, so that the customers will be more inclined to purchase the products. 

# Conclusion

In conclusion, Association Rules can have many different usage for an e-commerce shop owner. It is able to identify patterns in the purchases of customers, that would have otherwise been overlooked. By employing this model, e-commerce shop owners can easily and creatively improve their sales. 